# Random-Subspace SVM — Demo Experiment

This notebook shows a reproducible, 5-minute demo comparing a Random‑Subspace SVM to a baseline SVM.
Run all cells to reproduce results.

In [ ]:
# %pip install -q numpy scipy scikit-learn matplotlib
import numpy as np
from time import perf_counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
from sklearn.svm import SVC, LinearSVC
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from svm_random_subspaces import RandomSubspaceSVM


In [ ]:
RANDOM_SEED = 7
X, y = make_classification(n_samples=2000, n_features=50, n_informative=10,
                           n_redundant=10, class_sep=1.2, flip_y=0.02,
                           random_state=RANDOM_SEED)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_SEED, stratify=y
)
X_train.shape, X_test.shape


In [ ]:
baseline = Pipeline([
    ('scaler', StandardScaler(with_mean=True)),
    ('clf', SVC(kernel='rbf', C=1.0, gamma='scale'))
])
rssvm = RandomSubspaceSVM(base_estimator=LinearSVC(dual=False), M=25, k=20,
                          voting='soft', random_state=RANDOM_SEED, scale=True)
import time
t0=time.time(); baseline.fit(X_train, y_train); t_baseline=time.time()-t0
t0=time.time(); rssvm.fit(X_train, y_train);     t_rssvm=time.time()-t0
yhat_base = baseline.predict(X_test)
yhat_rss  = rssvm.predict(X_test)
print(f"Baseline SVM (RBF): acc={accuracy_score(y_test, yhat_base):.3f} time={t_baseline*1e3:.0f}ms")
print(f"Random‑Subspace SVM: acc={accuracy_score(y_test, yhat_rss):.3f} time={t_rssvm*1e3:.0f}ms")


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ConfusionMatrixDisplay.from_predictions(y_test, yhat_base, ax=axes[0], colorbar=False)
axes[0].set_title('Baseline SVM (RBF)')
ConfusionMatrixDisplay.from_predictions(y_test, yhat_rss, ax=axes[1], colorbar=False)
axes[1].set_title('Random‑Subspace SVM')
plt.tight_layout(); plt.show()


**Next steps**: vary `M`, `k`, try RBF base SVMs, add datasets, and log seeds for full reproducibility.